In [1]:
from final_utils import *
random.seed(5)

In [2]:
SC_CE_KLD = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
root_path = generate_root_path() 

In [4]:
indxs = np.load('variant1_label_9744.npy')
def take_from_vector(feature_vector, indxs ):    
#     return feature_vector[indxs]
    return np.take(feature_vector, indxs)

### Loading alm data features

#### Loading  multigap features from .json

In [5]:
# main_path = root_path + 'Data/splitted/train/'
# features_bad_list = []
# features_bad_list_i = []
# features_good1 = []
# feats = 'original_PCA_8464_auto'
    
# for i in range(7):
#     alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
#     bad_data = json.load(alm_train_bad)
#     for data in bad_data:
#         feat_path = main_path + f'features/mg/{feats}/' + data['feature']
#         features_bad_list_i.append(np.load(feat_path))
#     features_bad_list.append(features_bad_list_i)
#     features_bad_list_i = []
        
# alm_train_good = open(f'{main_path}/data_good1.json')
# good_data = json.load(alm_train_good)
# for data in good_data:
#     feat_path = main_path + f'features/mg/{feats}/' + data['feature']
#     features_good1.append(np.load(feat_path))
    
# for i in range(7):
#     features_bad_list[i] = np.squeeze(np.array(features_bad_list[i]))
# features_good1 = np.squeeze(np.array(features_good1))
   
# # Generating static validation data
# features_bad_list[0], features_bad1_val = extract_static_val_data(features_bad_list[0], perc = 0.11)
# features_good1, features_good1_val = extract_static_val_data(features_good1, perc = 0.11)

# bad = features_bad_list

#### Loading cnn features + multigap features from .json

In [6]:
main_path = root_path + 'Data/splitted/train/'
features_bad_list = []
features_bad_list_i = []
features_good1 = []
feats_MG = 'original' 
feats_CNN = 'border_600x600'
# feats_CNN_MG_PCA = 'cnn_mg_concat/pca_9744_auto'
cnn = 'cnn_efficientnet_b7'
    
for i in range(7):
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    for data in bad_data:
        feat_path_1 = main_path + f'features/mg/{feats_MG}/' + data['feature']
        feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
        connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load(feat_path_2))))
        connected = take_from_vector(connected, indxs) # must be commented later
#         connected = main_path + f'features/{feats_CNN_MG_PCA}/' + data['feature']
#         connected = np.squeeze(np.load(connected))
        features_bad_list_i.append(connected)
        
    features_bad_list.append(features_bad_list_i)
    features_bad_list_i = []
      
alm_train_good = open(f'{main_path}/data_good1.json')
good_data = json.load(alm_train_good)
for data in good_data:
    feat_path_1 = main_path + f'features/mg/{feats_MG}/' + data['feature']
    feat_path_2 = main_path + f'features/{cnn}/{feats_CNN}/' + data['feature']
    connected = np.concatenate((np.squeeze(np.load(feat_path_1)), np.squeeze(np.load((feat_path_2)))))
    connected = take_from_vector(connected, indxs) # must be commented later
#     connected = main_path + f'features/{feats_CNN_MG_PCA}/' + data['feature']
#     connected = np.squeeze(np.load(connected))
    features_good1.append(connected)
    
for i in range(7):
    features_bad_list[i] = np.squeeze(np.array(features_bad_list[i]))
features_good1 = np.squeeze(np.array(features_good1))
   
# Generating static validation data
features_bad_list[0], features_bad1_val = extract_static_val_data(features_bad_list[0], perc = 0.11)
features_good1, features_good1_val = extract_static_val_data(features_good1, perc = 0.11)

bad = features_bad_list

### Creating validation data

In [7]:
X_val = np.concatenate( (features_bad1_val, features_good1_val ) , axis=0 )
y_val = np.concatenate( (np.zeros(len(features_bad1_val)), np.ones(len(features_good1_val)) ), axis=0 )

### Some necessary functions

In [8]:
def plt_graphs(history):
    plt.plot(np.arange(2, len(history['loss'])+1), history['loss'][1:])
    plt.plot(np.arange(2, len(history['loss'])+1), history['val_loss'][1:])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
def save_history(history, name):
    np.savez(name, val_loss=np.array(history.history['val_loss']),
                       loss=np.array(history.history['loss']))
    
    
def calc_acc(model, weights_path, X_test, y_test, batch_size):
    '''
    Compares Max classes with targets, getting mean class precision
    '''
    model.load_weights(weights_path)
    y_pred = model.predict(X_test, batch_size=batch_size, verbose=0)
    acc = (np.argmax(y_pred,axis=-1) == y_test).mean()
    return acc

def lr_exp_decay(epoch, lr):
    k = 0.048
    return lr * np.exp(-k*epoch)



### Trainer


In [9]:
def trainer(model, data, weights_path, data_val,batch_size=128, epochs=30, learning_rate=0.03):
    X_train, X_test, y_train, y_test = data

    model.compile(loss=SC_CE_KLD,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                                  epsilon=1e-07, decay=0, amsgrad=False))
    model.load_weights(weights_path) 

    checkpoint = keras.callbacks.ModelCheckpoint(weights_path, 
                                                 monitor='val_loss', 
                                                 verbose=1, 
                                                 save_best_only=True, 
                                                 mode='min')
    
    
    schedule = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay, verbose=0)
    callbacks_list = [checkpoint, schedule]

    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        callbacks=callbacks_list,
                        validation_data = data_val)

    return history

### Creating and loading weights 

In [10]:
model = fc_model_softmax(input_num=9744)
weights_path_pretrained = f'models/Softmax/MG_CNN/model_fc_softmax_v1_9744_label.hdf5'
# model.save_weights(weights_path_pretrained) #if we want to cancel learning and start from 0
model.load_weights(weights_path_pretrained)
weights_path =  f'models/Softmax/MG_CNN/model_fc_softmax_v1_9744_label.hdf5'     
# model.save_weights(weights_path)

### Training

In [11]:
epochs = 15
batch_size = 128
learning_rate = 0.003

In [12]:
for i in range(len(bad)):
    print(bad[i].shape)
print(features_good1.shape)

(8899, 9744)
(10000, 9744)
(10000, 9744)
(9999, 9744)
(9998, 9744)
(9997, 9744)
(10717, 9744)
(10465, 9744)


In [14]:
data_val = (X_val, y_val)

i = 6
data = get_train_pairs(bad[i], features_good1, train_size=0.95, shuffle=True)
if True:
    history = trainer(model, data, weights_path, data_val, batch_size, epochs, learning_rate=learning_rate)
    acc = calc_acc(model, weights_path, data_val[0], data_val[1], batch_size)

    #save_history(history, f'histories/denoised_good_to_bad/model_fc_good_denoised_to_bad_02_08_bad{i+1}')

    print('----- Accuracy =', acc, ' -----')
    print('---Batch Train Done---')

train_data_bad shape (10717, 9744)
train_data_good shape (10465, 9744)
Epoch 1/15
157/158 [============================>.] - ETA: 0s - loss: 0.0720
Epoch 1: val_loss improved from inf to 0.07486, saving model to models/Softmax/MultiGap_CNN\model_fc_softmax_v1_9744_label.hdf5
158/158 [==============================] - 3s 17ms/step - loss: 0.0720 - val_loss: 0.0749 - lr: 0.0030
Epoch 2/15
157/158 [============================>.] - ETA: 0s - loss: 0.0600
Epoch 2: val_loss did not improve from 0.07486
158/158 [==============================] - 2s 14ms/step - loss: 0.0600 - val_loss: 0.0832 - lr: 0.0029
Epoch 3/15
157/158 [============================>.] - ETA: 0s - loss: 0.0500
Epoch 3: val_loss did not improve from 0.07486
158/158 [==============================] - 2s 14ms/step - loss: 0.0499 - val_loss: 0.0798 - lr: 0.0026
Epoch 4/15
157/158 [============================>.] - ETA: 0s - loss: 0.0436
Epoch 4: val_loss did not improve from 0.07486
158/158 [==============================] - 